# 收银台免密补贴EDA

## 增益模型 —— 分档

In [1]:
import datetime
import numpy as np
import pandas as pd
import joblib
import warnings
import logging
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import collections
import re
import copy
import xgboost as xgb
import lightgbm as lgb
import shap
import causalml
import statsmodels.api as sm

import utils
import propensity

from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from joblib import Parallel, delayed
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from supervised.automl import AutoML
from causalml.inference.meta import BaseSClassifier, BaseTClassifier, BaseXClassifier, BaseRClassifier

from causalml_visualize.visualize_ import plot_all

pd.set_option('display.max_columns', None)
pd.set_option('max_row', 500)
warnings.filterwarnings('ignore')
tqdm.pandas(desc='pandas bar')
logger = logging.getLogger('causalml')
logging.basicConfig(level=logging.INFO)
os.environ['NUMEXPR_MAX_THREADS'] = str(os.cpu_count())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/odin/jarretthan/anaconda3/envs/deepctr/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_194913/135023387.py", line 10, in <module>
    import seaborn as sns
  File "/home/odin/jarretthan/anaconda3/envs/deepctr/lib/python3.7/site-packages/seaborn/__init__.py", line 12, in <module>
    from .widgets import *  # noqa: F401,F403
  File "/home/odin/jarretthan/anaconda3/envs/deepctr/lib/python3.7/site-packages/seaborn/widgets.py", line 7, in <module>
    from ipywidgets import interact, FloatSlider, IntSlider
  File "/home/odin/jarretthan/anaconda3/envs/deepctr/lib/python3.7/site-packages/ipywidgets/__init__.py", line 25, in <module>
    from .widgets import *
  File "/home/odin/jarretthan/anaconda3/envs/deepctr/lib/python3.7/site-packages/ipywidgets/widgets/__init__.py", line 11, in <module>
    from .widget_bool import Checkbox, 

TypeError: object of type 'NoneType' has no len()

In [ ]:
xgb.__version__

In [ ]:
lgb.__version__

## data

In [ ]:
# df = pd.read_csv('data/no_secret_bt/sample_label_feature_fusion_20220206_20220227.txt', sep='\t', encoding='utf-8')
df = pd.read_csv('data/no_secret_bt/sample_label_feature_aspiration_20220206_20220227.txt', sep='\t', encoding='utf-8')

print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
df.groupby(by=['obs_dt'])['uid'].count()

In [ ]:
df.groupby(by=['coupon'])['uid'].count()

In [ ]:
df['label'].value_counts()

In [ ]:
80373 / 1080373

In [ ]:
df['obs_dt'] = pd.to_datetime(df['obs_dt'])
df.head()

In [ ]:
# utils.save_pickle(df, 'data/no_secret_bt/df_sample_label_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(df, 'data/no_secret_bt/df_sample_label_feature_aspiration_20220206_20220227.pickle')

## feature select

* 方差

In [ ]:
# df_all = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_fusion_20220206_20220227.pickle')
df_all = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_aspiration_20220206_20220227.pickle')

print(df_all.shape)
df_all.head()

In [ ]:
df_all_des = utils.df_des(df_all)
print(df_all_des.shape)
df_all_des.head()

In [ ]:
# df_all_des.to_csv('data/no_secret_bt/df_des_sample_label_feature_fusion_20220206_20220227.csv', encoding='utf-8')
df_all_des.to_csv('data/no_secret_bt/df_des_sample_label_feature_aspiration_20220206_20220227.csv', encoding='utf-8')

In [ ]:
list_feats_id_y = ['uid', 'obs_dt', 'coupon', 'label']
list_feats_x = [x for x in df_all.columns if x not in list_feats_id_y]
print(len(list_feats_x))
list_feats_x[:10]

In [ ]:
# utils.save_pickle(list_feats_x, 'data/no_secret_bt/list_feats/list_feats_x_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(list_feats_x, 'data/no_secret_bt/list_feats/list_feats_x_feature_aspiration_20220206_20220227.pickle')

In [ ]:
list_feats_x_std_0 = [x for x in df_all_des[df_all_des['std']==0].index]
print(len(list_feats_x_std_0))
list_feats_x_std_0[:10]

In [ ]:
# utils.save_pickle(list_feats_x_std_0, 'data/no_secret_bt/list_feats/list_feats_x_std_0_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(list_feats_x_std_0, 'data/no_secret_bt/list_feats/list_feats_x_std_0_feature_aspiration_20220206_20220227.pickle')

In [ ]:
list_feats_x_std = [x for x in list_feats_x if x not in list_feats_x_std_0]
print(len(list_feats_x_std))
list_feats_x_std[:10]

In [ ]:
# utils.save_pickle(list_feats_x_std, 'data/no_secret_bt/list_feats/list_feats_x_std_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(list_feats_x_std, 'data/no_secret_bt/list_feats/list_feats_x_std_feature_aspiration_20220206_20220227.pickle')

* F检验（treatment+X，分组加权平均）

In [ ]:
# list_feats_x_std = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_std_feature_fusion_20220206_20220227.pickle')
list_feats_x_std = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_std_feature_aspiration_20220206_20220227.pickle')

print(len(list_feats_x_std))
list_feats_x_std[:10]

In [ ]:
# copy causalml中的代码
# 修改应用于multi-treatment（多档金额），分组计算，取均值
def uplift_filter_F_one_feature(data, treatment_group, treatment_indicator, feature_name, y_name):     
    Y = data[y_name]
    X = data[[treatment_indicator, feature_name]]
    X = sm.add_constant(X)
    X['{}-{}'.format(treatment_indicator, feature_name)] = X[[treatment_indicator, feature_name]].product(axis=1)
    
    model = sm.OLS(Y, X)
    result = model.fit()
    
    F_test = result.f_test(np.array([0, 0, 0, 1]))
    F_test_result = pd.DataFrame({
        'feature': feature_name, 
        'p_value_{tg}'.format(tg=treatment_group): F_test.pvalue
    }, index=[0]).reset_index(drop=True)
    
    return F_test_result


def uplift_filter_F(data, treatment_name, control_group, treatments_group, features_name, y_name, 
                    treatments_group_num=None):
    '''
    data: pandas.DataFrame
    treatment_name: string
    control_group: string\int\double
    treatment_group: list[string\int\double]
    features_name: list[string]
    y_name: string
    '''
    
    f_test = pd.DataFrame({'feature': features_name})
    
    for treatment_group in treatments_group:
        data_tmp = data[data[treatment_name].isin([control_group, treatment_group])]
        data_tmp['treatment_indicator'] = 0
        data_tmp.loc[data_tmp[treatment_name]==treatment_group, 'treatment_indicator'] = 1
        
        list_f_test_tg = []
        try:
            with tqdm(features_name) as t:
                for feature_name in t:
                    f_test_tmp = uplift_filter_F_one_feature(data_tmp, treatment_group, 'treatment_indicator', feature_name, y_name)
                    list_f_test_tg.append(f_test_tmp)
        except KeyboardInterrupt:
            t.close()
            raise
        t.close()
        
        f_test_tg = pd.concat(list_f_test_tg, axis=0)
        f_test_tg.reset_index(drop=True, inplace=True)
        
        f_test = f_test.merge(f_test_tg, on='feature', how='left')
    
    if treatments_group_num is None:
        f_test['p_value_mean'] = f_test[[x for x in f_test.columns if x != 'feature']].mean(axis=1)
    else:
        f_test['p_value_mean'] = f_test[[x for x in f_test.columns if x != 'feature']].\
            agg(func=np.average, axis=1, weights=treatments_group_num)
    f_test.sort_values(by=['p_value_mean'], ascending=[True], inplace=True)
    
    return f_test

In [ ]:
control_group = 0.0
treatments_group = [0.2, 0.5, 1.0, 1.5, 1.8]
weights = [48304, 281640, 51025, 39244, 30853]
f_test = uplift_filter_F(df_all, 'coupon', control_group, treatments_group, list_feats_x_std, 'label', treatments_group_num=weights)
f_test.head()

In [ ]:
# f_test.to_csv('data/no_secret_bt/f_test/feature_fusion_20220206_20220227.csv', encoding='utf-8', index=False)
f_test.to_csv('data/no_secret_bt/f_test/feature_aspiration_20220206_20220227.csv', encoding='utf-8', index=False)

In [ ]:
list_feats_x_ft_no_select = f_test[(f_test['p_value_mean']>0.05)|(f_test['p_value_mean'].isna())]['feature'].values.tolist()
print(len(list_feats_x_ft_no_select))
list_feats_x_ft_no_select

In [ ]:
# utils.save_pickle(list_feats_x_ft_no_select, 'data/no_secret_bt/list_feats/list_feats_x_ht_no_select_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(list_feats_x_ft_no_select, 'data/no_secret_bt/list_feats/list_feats_x_ht_no_select_feature_aspiration_20220206_20220227.pickle')

In [ ]:
list_feats_x_ht = [x for x in list_feats_x_std if x not in list_feats_x_ft_no_select]
print(len(list_feats_x_ht))
list_feats_x_ht[:10]

In [ ]:
# utils.save_pickle(list_feats_x_ht, 'data/no_secret_bt/list_feats/list_feats_x_ht_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(list_feats_x_ht, 'data/no_secret_bt/list_feats/list_feats_x_ht_feature_aspiration_20220206_20220227.pickle')

* 模型
* 特征重要性
* PI方法，使用lgb+X-learner进行建模预测

In [ ]:
# list_feats_x_ht = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_ht_feature_fusion_20220206_20220227.pickle')
list_feats_x_ht = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_ht_feature_aspiration_20220206_20220227.pickle')

print(len(list_feats_x_ht))
list_feats_x_ht[:10]

In [ ]:
# id，treatment，X，y
df_sample_unique_id = df_all[['uid', 'obs_dt']]
df_sample_unique_treatment = df_all['coupon']
df_sample_unique_y = df_all['label']
df_sample_unique_X = df_all[list_feats_x_ht]

print(df_sample_unique_id.shape)
print(df_sample_unique_treatment.shape)
print(df_sample_unique_y.shape)
print(df_sample_unique_X.shape)

# utils.save_pickle(df_sample_unique_id, 'data/no_secret_bt/df_sample_unique_id_feature_fusion_20220206_20220227.pickle')
# utils.save_pickle(df_sample_unique_treatment, 'data/no_secret_bt/df_sample_unique_treatment_feature_fusion_20220206_20220227.pickle')
# utils.save_pickle(df_sample_unique_y, 'data/no_secret_bt/df_sample_unique_y_feature_fusion_20220206_20220227.pickle')
# utils.save_pickle(df_sample_unique_X, 'data/no_secret_bt/df_sample_unique_X_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(df_sample_unique_id, 'data/no_secret_bt/df_sample_unique_id_feature_aspiration_20220206_20220227.pickle')
utils.save_pickle(df_sample_unique_treatment, 'data/no_secret_bt/df_sample_unique_treatment_feature_aspiration_20220206_20220227.pickle')
utils.save_pickle(df_sample_unique_y, 'data/no_secret_bt/df_sample_unique_y_feature_aspiration_20220206_20220227.pickle')
utils.save_pickle(df_sample_unique_X, 'data/no_secret_bt/df_sample_unique_X_feature_aspiration_20220206_20220227.pickle')

In [ ]:
# df_sample_unique_id = utils.load_pickle('data/no_secret_bt/df_sample_unique_id_feature_fusion_20220206_20220227.pickle')
# df_sample_unique_treatment = utils.load_pickle('data/no_secret_bt/df_sample_unique_treatment_feature_fusion_20220206_20220227.pickle')
# df_sample_unique_y = utils.load_pickle('data/no_secret_bt/df_sample_unique_y_feature_fusion_20220206_20220227.pickle')
# df_sample_unique_X = utils.load_pickle('data/no_secret_bt/df_sample_unique_X_feature_fusion_20220206_20220227.pickle')
df_sample_unique_id = utils.load_pickle('data/no_secret_bt/df_sample_unique_id_feature_aspiration_20220206_20220227.pickle')
df_sample_unique_treatment = utils.load_pickle('data/no_secret_bt/df_sample_unique_treatment_feature_aspiration_20220206_20220227.pickle')
df_sample_unique_y = utils.load_pickle('data/no_secret_bt/df_sample_unique_y_feature_aspiration_20220206_20220227.pickle')
df_sample_unique_X = utils.load_pickle('data/no_secret_bt/df_sample_unique_X_feature_aspiration_20220206_20220227.pickle')

print(df_sample_unique_id.shape)
print(df_sample_unique_treatment.shape)
print(df_sample_unique_y.shape)
print(df_sample_unique_X.shape)

In [ ]:
df_sample_unique_treatment_map = df_sample_unique_treatment.map({0.0: 'control', 
                                                                 0.2: 'treatment_1', 
                                                                 0.5: 'treatment_2', 
                                                                 1.0: 'treatment_3', 
                                                                 1.5: 'treatment_4', 
                                                                 1.8: 'treatment_5'})
df_sample_unique_treatment_map

In [ ]:
# utils.save_pickle(df_sample_unique_treatment_map, 'data/no_secret_bt/df_sample_unique_treatment_map_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(df_sample_unique_treatment_map, 'data/no_secret_bt/df_sample_unique_treatment_map_feature_aspiration_20220206_20220227.pickle')

In [ ]:
# df_sample_unique_treatment_map = utils.load_pickle('data/no_secret_bt/df_sample_unique_treatment_map_feature_fusion_20220206_20220227.pickle')
df_sample_unique_treatment_map = utils.load_pickle('data/no_secret_bt/df_sample_unique_treatment_map_feature_aspiration_20220206_20220227.pickle')
df_sample_unique_treatment_map

In [ ]:
# 初始化倾向性得分（暂使用各treatment组占比（treatment+control），简单设置用于特征筛选）
dict_p = {}
dict_p['treatment_1'] = np.array([0.0713]*df_sample_unique_id.shape[0])
dict_p['treatment_2'] = np.array([0.3092]*df_sample_unique_id.shape[0])
dict_p['treatment_3'] = np.array([0.0750]*df_sample_unique_id.shape[0])
dict_p['treatment_4'] = np.array([0.0587]*df_sample_unique_id.shape[0])
dict_p['treatment_5'] = np.array([0.0467]*df_sample_unique_id.shape[0])
dict_p

In [ ]:
30853 / (629307+30853)

In [ ]:
# utils.save_pickle(dict_p, 'data/no_secret_bt/pi/dict_p_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(dict_p, 'data/no_secret_bt/pi/dict_p_feature_aspiration_20220206_20220227.pickle')

In [ ]:
# dict_p = utils.load_pickle('data/no_secret_bt/pi/dict_p_20220206_20220227.pickle')
dict_p = utils.load_pickle('data/no_secret_bt/pi/dict_p_feature_aspiration_20220206_20220227.pickle')
dict_p

In [ ]:
%%time
# X-learner-lgb
learner_x_lgb = BaseXClassifier(control_outcome_learner=lgb.LGBMClassifier(), 
                                treatment_outcome_learner=lgb.LGBMClassifier(), 
                                control_effect_learner=lgb.LGBMRegressor(), 
                                treatment_effect_learner=lgb.LGBMRegressor(), 
                                control_name='control')
pred_x = learner_x_lgb.fit_predict(df_sample_unique_X.values, df_sample_unique_treatment_map.values, df_sample_unique_y.values, p=dict_p)
# joblib.dump(learner_x_lgb, 'data/no_secret_bt/pi/estimator_pi_learner_x_lgb_feature_fusion_20220206_20220227.pickle')
joblib.dump(learner_x_lgb, 'data/no_secret_bt/pi/estimator_pi_learner_x_lgb_feature_aspiration_20220206_20220227.pickle')

In [ ]:
df_auuc_x = pd.DataFrame({'model': ['X-learner-lgb']})
for group in learner_x_lgb.t_groups:
    print(group)
    auuc = plot_all(cate=pred_x, treatment_groups=sorted(list(learner_x_lgb.t_groups)), treatment_test=df_sample_unique_treatment_map.values, 
                    y_test=df_sample_unique_y.values, cost_test=df_sample_unique_treatment.values, title='X-learner-LGB-p multi-treatment-{g} uplift curve'.format(g=group), 
                    select_treatment_group=group)
    df_auuc_x['auuc_'+group] = [auuc]

treatments_group_num = [48304, 281640, 51025, 39244, 30853]
df_auuc_x['auuc_mean'] = df_auuc_x[[x for x in df_auuc_x.columns if x != 'model']].\
    agg(func=np.average, axis=1, weights=treatments_group_num)

In [ ]:
df_auuc_x

In [ ]:
# learner_x_lgb = joblib.load('data/no_secret_bt/pi/estimator_pi_learner_x_lgb_feature_fusion_20220206_20220227.pickle')
learner_x_lgb = joblib.load('data/no_secret_bt/pi/estimator_pi_learner_x_lgb_feature_aspiration_20220206_20220227.pickle')
learner_x_lgb

In [ ]:
pred_x = learner_x_lgb.predict(df_sample_unique_X.values, p=dict_p)
pred_x

In [ ]:
# utils.save_pickle(pred_x, 'data/no_secret_bt/pi/estimator_pi_learner_x_lgb_pred_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(pred_x, 'data/no_secret_bt/pi/estimator_pi_learner_x_lgb_pred_feature_aspiration_20220206_20220227.pickle')

In [ ]:
# pred_x = utils.load_pickle('data/no_secret_bt/pi/estimator_pi_learner_x_lgb_pred_feature_fusion_20220206_20220227.pickle')
pred_x = utils.load_pickle('data/no_secret_bt/pi/estimator_pi_learner_x_lgb_pred_feature_aspiration_20220206_20220227.pickle')
print(pred_x.shape)
pred_x

In [ ]:
%%time
dict_feats_imp_pi = learner_x_lgb.get_importance(X=df_sample_unique_X.values, tau=pred_x, method='permutation')
# utils.save_pickle(dict_feats_imp_pi, 'data/no_secret_bt/pi/pi_learner_x_lgb_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(dict_feats_imp_pi, 'data/no_secret_bt/pi/pi_learner_x_lgb_feature_aspiration_20220206_20220227.pickle')
dict_feats_imp_pi

In [ ]:
# dict_feats_imp_pi = utils.load_pickle('data/no_secret_bt/pi/pi_learner_x_lgb_feature_fusion_20220206_20220227.pickle')
dict_feats_imp_pi = utils.load_pickle('data/no_secret_bt/pi/pi_learner_x_lgb_feature_aspiration_20220206_20220227.pickle')
dict_feats_imp_pi

In [ ]:
df_pi = pd.DataFrame({'feature': list_feats_x_ht, 
                      'feature_index': ['Feature_{i}'.format(i=(str(x) if x>=100 else '0'+str(x) if x>=10 else '00'+str(x))) 
                                        for x in range(len(list_feats_x_ht))]
                     })

for x in sorted(list(dict_feats_imp_pi.keys())):
    df_pi_tg = pd.DataFrame(dict_feats_imp_pi[x], columns=['permutation_importance_{}'.format(x)]).\
        reset_index().sort_values(by=['index'])
    df_pi_tg.rename(columns={'index': 'feature_index'}, inplace=True)
    df_pi = df_pi.merge(df_pi_tg, on='feature_index', how='left')

treatments_group_num = [48304, 281640, 51025, 39244, 30853]
df_pi['permutation_importance_mean'] = df_pi[[x for x in df_pi.columns if x not in ['feature', 'feature_index']]].\
    agg(func=np.average, axis=1, weights=treatments_group_num)
df_pi.sort_values(by=['permutation_importance_mean'], ascending=[False], inplace=True)
df_pi

In [ ]:
df_pi[['permutation_importance_mean']].quantile([x/10 for x in range(11)]).T

In [ ]:
list_feats_x_pi_no_select = df_pi[(df_pi['permutation_importance_mean']<=0.0)]['feature'].values.tolist()
print(len(list_feats_x_pi_no_select))
list_feats_x_pi_no_select[:10]

In [ ]:
# utils.save_pickle(list_feats_x_pi_no_select, 'data/no_secret_bt/list_feats/list_feats_x_pi_no_select_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(list_feats_x_pi_no_select, 'data/no_secret_bt/list_feats/list_feats_x_pi_no_select_feature_aspiration_20220206_20220227.pickle')

In [ ]:
list_feats_x_pi = [x for x in list_feats_x_ht if x not in list_feats_x_pi_no_select]
print(len(list_feats_x_pi))
list_feats_x_pi[:10]

In [ ]:
# utils.save_pickle(list_feats_x_pi, 'data/no_secret_bt/list_feats/list_feats_x_pi_feature_fusion_20220206_20220227.pickle')
utils.save_pickle(list_feats_x_pi, 'data/no_secret_bt/list_feats/list_feats_x_pi_feature_aspiration_20220206_20220227.pickle')

## feature merge

In [ ]:
# fusion
list_feats_x_fusion = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_pi_feature_fusion_20220206_20220227.pickle')
print(len(list_feats_x_fusion))
list_feats_x_fusion[:10]

In [ ]:
df_fusion_all = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_fusion_20220206_20220227.pickle')
print(df_fusion_all.shape)
df_fusion_all.head()

In [ ]:
df_fusion = df_fusion_all[['uid', 'obs_dt', 'coupon', 'label']+list_feats_x_fusion]
print(df_fusion.shape)
df_fusion.head()

In [ ]:
# aspiration
list_feats_x_aspiration = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_pi_feature_aspiration_20220206_20220227.pickle')
print(len(list_feats_x_aspiration))
list_feats_x_aspiration[:10]

In [ ]:
df_aspiration_all = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_aspiration_20220206_20220227.pickle')
print(df_aspiration_all.shape)
df_aspiration_all.head()

In [ ]:
df_aspiration = df_aspiration_all[['uid', 'obs_dt', 'coupon', 'label']+list_feats_x_aspiration]
print(df_aspiration.shape)
df_aspiration.head()

In [ ]:
%%time
df_union = df_fusion.merge(df_aspiration, on=['uid', 'obs_dt', 'coupon', 'label'], how='inner')
print(df_union.shape)
df_union.head()

In [ ]:
df_union_des = utils.df_des(df_union)
print(df_union_des.shape)
df_union_des.head()

In [ ]:
df_union_des[df_union_des['Miss Percent(%)']>0]

In [ ]:
utils.save_pickle(df_union, 'data/no_secret_bt/df_sample_label_feature_union_20220206_20220227.pickle')

In [ ]:
list_feats_x_union = list_feats_x_fusion + list_feats_x_aspiration
print(len(list_feats_x_union))
list_feats_x_union[:10]

In [ ]:
utils.save_pickle(list_feats_x_union, 'data/no_secret_bt/list_feats/list_feats_x_feature_union_20220206_20220227.pickle')

## model

In [ ]:
df_union = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_union_20220206_20220227.pickle')
print(df_union.shape)
df_union.head()

In [ ]:
list_feats_x_union = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_feature_union_20220206_20220227.pickle')
print(len(list_feats_x_union))
list_feats_x_union[:10]

In [ ]:
# id，treatment，X，y
df_uid_obsDate = df_union[['uid', 'obs_dt']]
df_treatment = df_union['coupon']
df_y = df_union['label']
df_X = df_union[list_feats_x_union]

print(df_uid_obsDate.shape)
print(df_treatment.shape)
print(df_y.shape)
print(df_X.shape)

utils.save_pickle(df_uid_obsDate, 'data/no_secret_bt/df_sample_label_feature_union_uid_obsDate_20220206_20220227.pickle')
utils.save_pickle(df_treatment, 'data/no_secret_bt/df_sample_label_feature_union_treatment_20220206_20220227.pickle')
utils.save_pickle(df_y, 'data/no_secret_bt/df_sample_label_feature_union_y_20220206_20220227.pickle')
utils.save_pickle(df_X, 'data/no_secret_bt/df_sample_label_feature_union_X_20220206_20220227.pickle')

In [ ]:
df_uid_obsDate = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_union_uid_obsDate_20220206_20220227.pickle')
df_treatment = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_union_treatment_20220206_20220227.pickle')
df_y = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_union_y_20220206_20220227.pickle')
df_X = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_union_X_20220206_20220227.pickle')

print(df_uid_obsDate.shape)
print(df_treatment.shape)
print(df_y.shape)
print(df_X.shape)

In [ ]:
df_treatment_map = df_treatment.map({0.0: 'control', 
                                     0.2: 'treatment_1', 
                                     0.5: 'treatment_2', 
                                     1.0: 'treatment_3', 
                                     1.5: 'treatment_4', 
                                     1.8: 'treatment_5'})
df_treatment_map

In [ ]:
# 初始化倾向性得分（暂使用各treatment组占比（treatment+control），简单设置用于特征筛选）
dict_p = {}
dict_p['treatment_1'] = np.array([0.0713]*df_uid_obsDate.shape[0])
dict_p['treatment_2'] = np.array([0.3092]*df_uid_obsDate.shape[0])
dict_p['treatment_3'] = np.array([0.0750]*df_uid_obsDate.shape[0])
dict_p['treatment_4'] = np.array([0.0587]*df_uid_obsDate.shape[0])
dict_p['treatment_5'] = np.array([0.0467]*df_uid_obsDate.shape[0])
dict_p

In [ ]:
%%time
# X-learner-LGB
learner_x_lgb = BaseXClassifier(control_outcome_learner=lgb.LGBMClassifier(), 
                                treatment_outcome_learner=lgb.LGBMClassifier(), 
                                control_effect_learner=lgb.LGBMRegressor(), 
                                treatment_effect_learner=lgb.LGBMRegressor(), 
                                control_name='control')
learner_x_lgb.fit(df_X.values, df_treatment_map.values, df_y.values, p=dict_p)
joblib.dump(learner_x_lgb, 'data/model/no_secret_bt_20220206_20220227_union_all_XLearnerLgb.pickle')

In [ ]:
learner_x_lgb = joblib.load('data/model/no_secret_bt_20220206_20220227_union_all_XLearnerLgb.pickle')
learner_x_lgb

In [ ]:
pred_x = learner_x_lgb.predict(df_X.values, p=dict_p)
utils.save_pickle(pred_x, 'data/no_secret_bt/df_sample_label_feature_union_pred_20220206_20220227.pickle.pickle')
pred_x

In [ ]:
df_auuc_x = pd.DataFrame({'model': ['X-learner-lgb']})
for group in learner_x_lgb.t_groups:
    print(group)
    auuc = plot_all(cate=pred_x, treatment_groups=sorted(list(learner_x_lgb.t_groups)), treatment_test=df_treatment_map.values, 
                    y_test=df_y.values, cost_test=df_treatment.values, title='X-learner-LGB-p multi-treatment-{g} uplift curve'.format(g=group), 
                    select_treatment_group=group)
    df_auuc_x['auuc_'+group] = [auuc]

treatments_group_num = [48304, 281640, 51025, 39244, 30853]
df_auuc_x['auuc_mean'] = df_auuc_x[[x for x in df_auuc_x.columns if x != 'model']].\
    agg(func=np.average, axis=1, weights=treatments_group_num)

In [ ]:
df_auuc_x

## feature importance

In [ ]:
df_X = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_union_X_20220206_20220227.pickle')
pred_x = utils.load_pickle('data/no_secret_bt/df_sample_label_feature_union_pred_20220206_20220227.pickle')

print(df_X.shape)
print(pred_x.shape)

In [ ]:
learner_x_lgb = joblib.load('data/model/no_secret_bt_20220206_20220227_union_XleanerLgb.pickle')
learner_x_lgb

In [ ]:
%%time
dict_feats_imp = learner_x_lgb.get_importance(X=df_X.values, tau=pred_x, method='permutation')
utils.save_pickle(dict_feats_imp, 'data/no_secret_bt/feature_importance_union_20220206_20220227.pickle')
dict_feats_imp

In [ ]:
list_feats_x_union = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_feature_union_20220206_20220227.pickle')
print(len(list_feats_x_union))
list_feats_x_union[:10]

In [ ]:
df_fi = pd.DataFrame({'feature': list_feats_x_union, 
                      'feature_index': ['Feature_{i}'.format(i=(str(x) if x>=100 else '0'+str(x) if x>=10 else '00'+str(x))) 
                                        for x in range(len(list_feats_x_union))]
                     })

for x in sorted(list(dict_feats_imp.keys())):
    df_fi_tg = pd.DataFrame(dict_feats_imp[x], columns=['permutation_importance_{}'.format(x)]).\
        reset_index().sort_values(by=['index'])
    df_fi_tg.rename(columns={'index': 'feature_index'}, inplace=True)
    df_fi = df_fi.merge(df_fi_tg, on='feature_index', how='left')

treatments_group_num = [48304, 281640, 51025, 39244, 30853]
df_fi['permutation_importance_mean'] = df_fi[[x for x in df_fi.columns if x not in ['feature', 'feature_index']]].\
    agg(func=np.average, axis=1, weights=treatments_group_num)
df_fi.sort_values(by=['permutation_importance_mean'], ascending=[False], inplace=True)
df_fi

In [ ]:
df_fi[['permutation_importance_mean']].quantile([x/10 for x in range(11)]).T

In [ ]:
df_fi.to_csv('data/no_secret_bt/feature_importance_union_20220206_20220227_uplift.csv', encoding='utf-8', index=False)

In [ ]:
plt.rcParams['font.sans-serif'] = ['simhei']
plt.rcParams['font.serif'] = ['simhei']
plt.rcParams['axes.unicode_minus'] = False
sns.set(font_scale=1.5)
sns.set_style('darkgrid', {'font.sans-serif':['simhei', 'Droid Sans Fallback']})

plt.rcParams['figure.figsize'] = (12.0, 8.0)
fig, axes = plt.subplots(1, 1)
sns.barplot(x='permutation_importance_mean', y='feature', data=df_fi.head(25), ax=axes)
axes.set_title('Top-25 importance features')

plt.show()

In [ ]:
df_fi[['feature', 'permutation_importance_mean']].head(25)

In [ ]:
df_fi = pd.read_csv('data/no_secret_bt/feature_importance_union_20220206_20220227_uplift.csv', encoding='utf-8')
print(df_fi.shape)
df_fi.head()

In [ ]:
list_feats_x_pi_no_select = df_fi[(df_fi['permutation_importance_mean']<0.00001)]['feature'].values.tolist()
print(len(list_feats_x_pi_no_select))
list_feats_x_pi_no_select[:10]

In [ ]:
utils.save_pickle(list_feats_x_pi_no_select, 'data/no_secret_bt/list_feats/list_feats_x_pi_no_select_feature_union_20220206_20220227.pickle')

In [ ]:
list_feats_x_pi = [x for x in list_feats_x_union if x not in list_feats_x_pi_no_select]
print(len(list_feats_x_pi))
list_feats_x_pi[:10]

In [ ]:
utils.save_pickle(list_feats_x_pi, 'data/no_secret_bt/list_feats/list_feats_x_pi_feature_union_20220206_20220227.pickle')

## 全量数据拉取sql处理

In [ ]:
def del_sql_coalesce(sql_file_path, special_words, sql_file_path_new):
    with open(sql_file_path, 'r') as fi:
        list_sql = fi.readlines()
        
        str_pattern = r'|'.join(['( '+x+', )' for x in special_words])
        str_pattern_end = r'|'.join(['( '+x+' )' for x in special_words])
        str_pattern_from = r'^from '
        pattern = re.compile(str_pattern)
        pattern_end = re.compile(str_pattern_end)
        pattern_from = re.compile(str_pattern_from)
        
        list_sql_valid = []
        list_sql_valid_front = []
        list_sql_valid_back = []
        
        index_front = 0
        for i in range(len(list_sql)):
            if re.search(pattern_from, list_sql[i]):
                index_front = i
                break
        list_sql_valid_back = list_sql[index_front+1:]
        
        try:
            with tqdm(list(range(len(list_sql[:index_front])))) as t:
                for i in t:
                    if i == index_front - 1:
                        if not re.search(str_pattern_end, list_sql[i]):
                            list_sql_valid_front.append(list_sql[i])
                    else:
                        if not re.search(pattern, list_sql[i]):
                            list_sql_valid_front.append(list_sql[i])
        except KeyboardInterrupt:
            t.close()
            raise
        t.close()
        list_sql_valid = list_sql_valid_front + [list_sql[index_front]] + list_sql_valid_back
        
        with open(sql_file_path_new, 'w') as fo:
            fo.writelines(list_sql_valid)
            

def del_sql_select_origin(sql_file_path, special_words, sql_file_path_new):
    with open(sql_file_path, 'r') as fi:
        list_sql = fi.readlines()
        
        str_pattern_1 = r'|'.join(['(\t'+x+', )' for x in special_words])
        str_pattern_2 = r'|'.join(['(    '+x+', )' for x in special_words])
        str_pattern_3 = r'|'.join(['(\t'+x+' )' for x in special_words])
        str_pattern_4 = r'|'.join(['(    '+x+' )' for x in special_words])
        str_pattern_from = r'^from '
        pattern_1 = re.compile(str_pattern_1)
        pattern_2 = re.compile(str_pattern_2)
        pattern_3 = re.compile(str_pattern_3)
        pattern_4 = re.compile(str_pattern_4)
        pattern_from = re.compile(str_pattern_from)
        
        list_sql_valid = []
        list_sql_valid_front = []
        list_sql_valid_back = []
        
        index_front = 0
        for i in range(len(list_sql)):
            if re.search(pattern_from, list_sql[i]):
                index_front = i
                break
        list_sql_valid_front = list_sql[:index_front]
        
        try:
            with tqdm(list(range(len(list_sql[index_front+1:])))) as t:
                for i in t:
                    if not re.search(pattern_1, list_sql[index_front+1+i]) \
                        and not re.search(pattern_2, list_sql[index_front+1+i]) \
                        and not re.search(pattern_3, list_sql[index_front+1+i]) \
                        and not re.search(pattern_4, list_sql[index_front+1+i]):
                        list_sql_valid_back.append(list_sql[index_front+1+i])
        except KeyboardInterrupt:
            t.close()
            raise
        t.close()
        list_sql_valid = list_sql_valid_front + [list_sql[index_front]] + list_sql_valid_back
        
        with open(sql_file_path_new, 'w') as fo:
            fo.writelines(list_sql_valid)
            
            
def correct_end(sql_file_path, sql_file_path_new):
    with open(sql_file_path, 'r') as fi:
        list_sql = fi.readlines()
        
        str_pattern = r'from '
        pattern = re.compile(str_pattern)
        
        try:
            with tqdm(list(range(len(list_sql)))) as t:
                for i in t:
                    if re.search(pattern, list_sql[i]):
                        if list_sql[i-1][-3:-1] == ', ':
                            list_sql[i-1] = list_sql[i-1][:-3] + list_sql[i-1][-2:]
        except KeyboardInterrupt:
            t.close()
            raise
        t.close()
        
        with open(sql_file_path_new, 'w') as fo:
            fo.writelines(list_sql)

In [ ]:
list_feats_x = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_pi_feature_union_20220206_20220227.pickle')
print(len(list_feats_x))
list_feats_x[:10]

In [ ]:
# list_feats_x_all = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_feature_fusion_20220206_20220227.pickle')
list_feats_x_all = utils.load_pickle('data/no_secret_bt/list_feats/list_feats_x_feature_aspiration_20220206_20220227.pickle')
print(len(list_feats_x_all))
list_feats_x_all[:10]

In [ ]:
list_feats_x_del = [x for x in list_feats_x_all if x not in list_feats_x]
print(len(list_feats_x_del))
list_feats_x_del[:10]

In [ ]:
1821-1568

In [ ]:
4459-4123

In [ ]:
253+336

In [ ]:
# sql_file_path = 'sample_label_feature_fusion.sql'
sql_file_path = 'sample_label_feature_aspiration.sql'
special_words = list_feats_x_del
# sql_file_path_new = 'sample_label_feature_fusion_new_uplift.sql'
sql_file_path_new = 'sample_label_feature_aspiration_new_uplift.sql'

del_sql_coalesce(sql_file_path, special_words, sql_file_path_new)
del_sql_select_origin(sql_file_path_new, special_words, sql_file_path_new)
correct_end(sql_file_path_new, sql_file_path_new)